<a href="https://colab.research.google.com/github/mrchapagain/food_ontology/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here i try to explore different dataset to look if they can be connected, linked or somehow can talk to eachother.**
*   *Data by Frederik*
*   *Frida dataset of Nutrient content of food items*
*   *Reciept from COOP store*
*   *Reciept from my personal Storebox account*
*   *link from TradeSync data from GS1*



In [24]:
import pandas as pd
import os

# Data by Frederik

In [54]:
path="https://github.com/mrchapagain/food_ontology/raw/main/cpd_summary.csv"

data= pd.read_csv(path, sep= ";", index_col=0, encoding='latin-1')
data.index.unique()

Index([                'PANT',               'Diesel',        'FISKEAFDELING',
                  'Blyfri 95',    'ØKO BANANER, LØSE',         'BANANER,LØSE',
              'Blyfri 95 E10',              'BANANER',           'FLASKEPANT',
            'BETJENT FISK KG',
       ...
       'AAS MUNDBIND TYPE II',      '3F-HYLDEKNÆGT M',       'SKAGERAK BØJLE',
           'SMÅUDB. U/100 KR', 'SODASTREAM OMBYTNING',  'TECHNIC 42073 BASH!',
             'TEKSTILLEDNING', 'PINOTEX SUPERDEC SOR',  'UKRUDTSDUG BASIC 1X',
            'BR MÆRKER RETUR'],
      dtype='object', name='itemname', length=65449)

# Frida dataset

In [ ]:
path="https://github.com/mrchapagain/food_ontology/raw/main/FridaNutrientData.xlsx"

data= pd.read_csv(path, sep= ";", index_col=0, encoding='latin-1')
data.index.unique()

# Reciept from COOP store

In [ ]:
path="https://github.com/mrchapagain/food_ontology/raw/main/cpd_summary.csv"

data= pd.read_csv(path, sep= ";", index_col=0, encoding='latin-1')
data.index.unique()

# TradeSync data from GS1 

# Recipt from my personal Storebox account